<a href="https://colab.research.google.com/github/McKingN/PROJET_DDEFI_2025/blob/main/projet_data_ddefi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git config --global user.email "clabessim@gmail.com"
!git config --global user.name "McKingN"
!git clone https://github.com/Data-project-team-ecm/PROJET_DDEFI_2025.git


Cloning into 'PROJET_DDEFI_2025'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 8 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (8/8), done.
Resolving deltas: 100% (1/1), done.
